# AudioModel Testing Notebook 

Testing to see if the code I compiled from that guy works

### Thoughts:
* I think we will want to use weights and biases or tensorboard to keep track of the model's progress. 
    * Need to get a solid frame work for improvement set up, so the stuff we are doing isn't so random after all.
    * First we need to find a solid architecture, next we will find the best hyper parameters for our model. 

In [3]:
import wandb

In [4]:
wandb.login() # relogin=True

AttributeError: module 'wandb' has no attribute 'login'

In [ ]:
from AudioDataset import * 
from AudioModel import * 
from train import * 

from torch.utils.data import DataLoader


In [ ]:
# BATCH_SIZE = 128
# EPOCHS = 10
# LEARNING_RATE = 0.001

# AUDIO_DIR = '../data/Crema'
# SAMPLE_RATE = 16000
# NUM_SAMPLES = 22050


In [ ]:
hyperparameters = dict(
    epochs=10,
    batch_size=64, 
    learning_rate=0.001, 
    audio_dir='../data/Crema',
    sample_rate=16000,
    num_samples=22050, 
    n_fft=1024, 
    hop_length=512, 
    n_mels=64, 
    model='cnn', 
    dataset='crema'
)

#### TODO: add a `make` function! https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb#scrollTo=I87OUqkWvBKq

In [ ]:
with wandb.init(project='simple-melspectogram-cnn', config=hyperparameters):
    config=wandb.config
    
    if torch.cuda.is_available():
            device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")

    # instantiating our dataset object and create data loader
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=config.sample_rate,
        n_fft=config.n_fft,
        hop_length=config.hop_length,
        n_mels=config.n_mels
    )

    audio_dataset = AudioEmotionDataset(config.audio_dir,
                            mel_spectrogram,
                            config.sample_rate,
                            config.num_samples,
                            device)

    train_dataloader = create_data_loader(audio_dataset, config.batch_size)

    # construct model and assign it to device
    cnn = CNNNetwork().to(device)
    print(cnn)

    # initialise loss funtion + optimiser
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(cnn.parameters(),
                                 lr=config.learning_rate)
    
    train(cnn, train_dataloader, loss_fn, optimiser, device, config.epochs)

    # save model
    torch.save(cnn.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")

Using cuda
CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=6, bias=True)
  (softmax): Softmax(dim=1)
)



  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1
Loss after 01536 examples: 1.649
Loss after 03136 examples: 1.619
Loss after 04736 examples: 1.565
Loss after 06336 examples: 1.544



 10%|█         | 1/10 [01:10<10:30, 70.04s/it]

loss: 1.7439117431640625
---------------------------
Epoch 2
Loss after 07890 examples: 1.589
Loss after 09490 examples: 1.684
Loss after 11090 examples: 1.596
Loss after 12690 examples: 1.659
